In [26]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import regexp_tokenize
from nltk.stem.porter import PorterStemmer
import time
from scipy.sparse import csr_matrix
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

In [27]:
def clean_missing(df):
    df["category_name"] = df.loc[:,"category_name"].replace(np.NaN, "")
    df["brand_name"] = df.loc[:,"brand_name"].replace(np.NaN, "")
    df["item_description"] = df.loc[:,"item_description"].replace(np.NaN, "")
    return df

def split_cat(text):
    if text.count('/') > 1:
        return text.split("/")
    else:
        return (["No Label", "No Label", "No Label"])

def transform_category_name(df):
    df.loc[:,'general_cat'], df.loc[:,'subcat_1'], df.loc[:,'subcat_2'] = \
    zip(*df['category_name'].apply(lambda x: split_cat(x)))
    return df

def stem_tokenize(text, stop_words=[]):
    stemmer = PorterStemmer()
    tokens = regexp_tokenize(text, pattern=r"[A-Za-z]\w+")
    tokens_wo_sw = [x for x in tokens if x not in stop_words and len(x) > 3]
    tokens_stemmed = [stemmer.stem(x) for x in tokens_wo_sw]
    return tokens_stemmed

def save_sparse_csr(filename,array):
    np.savez(filename,data = array.data ,indices=array.indices,
             indptr =array.indptr, shape=array.shape )

def load_sparse_csr(filename):
    loader = np.load(filename)
    return csr_matrix((  loader['data'], loader['indices'], loader['indptr']),
                         shape = loader['shape'])

In [28]:
data = pd.read_csv("input/train.tsv", sep='\t')
test = pd.read_csv("input/test.tsv", sep='\t')
msk = np.random.rand(len(data)) < 0.01
sample_data = data[msk]
sample_data.shape

(14454, 8)

In [29]:
train = sample_data

In [30]:
print(train.shape)
print(test.shape)

(14454, 8)
(693359, 7)


In [31]:
train = clean_missing(train)
test = clean_missing(test)

C:\Users\thaunayf\AppData\Local\Continuum\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\thaunayf\AppData\Local\Continuum\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\thaunayf\AppData\Local\Continuum\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a 

In [9]:
train = transform_category_name(train)
test = transform_category_name(test)

C:\Users\thaunayf\AppData\Local\Continuum\Anaconda2\envs\py36\lib\site-packages\pandas\core\indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\thaunayf\AppData\Local\Continuum\Anaconda2\envs\py36\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [ ]:
y_train = np.log1p(train['price'].as_matrix())

In [ ]:
X_train = pd.get_dummies(train[['item_condition_id', 'shipping']]).as_matrix()
X_test =pd.get_dummies(test[['item_condition_id', 'shipping']]).as_matrix()


tfidf_vectorizer = TfidfVectorizer(tokenizer=stem_tokenize, decode_error='ignore', strip_accents='unicode', max_df=0.8, min_df=0.001)
tfidf_vectorizer.fit(train["category_name"])
X_train_cn = tfidf_vectorizer.transform(train["category_name"])
X_test_cn = tfidf_vectorizer.transform(test["category_name"])

X_train = np.concatenate((X_train, X_train_cn.toarray()), axis=1)
X_test = np.concatenate((X_test, X_test_cn.toarray()), axis=1)



tfidf_vectorizer = TfidfVectorizer(tokenizer=stem_tokenize, decode_error='ignore', strip_accents='unicode', max_df=0.8, min_df=0.001)
tfidf_vectorizer.fit(train["brand_name"])
X_train_bn = tfidf_vectorizer.transform(train["brand_name"])
X_test_bn = tfidf_vectorizer.transform(test["brand_name"])
    
X_train = np.concatenate((X_train, X_train_bn.toarray()), axis=1)
X_test = np.concatenate((X_test, X_test_bn.toarray()), axis=1)

tfidf_vectorizer = TfidfVectorizer(tokenizer=stem_tokenize, decode_error='ignore', strip_accents='unicode', max_df=0.8, min_df=0.001)
tfidf_vectorizer.fit(train["item_description"])
X_train_id = tfidf_vectorizer.transform(train["item_description"])
X_test_id = tfidf_vectorizer.transform(test["item_description"])

X_train = np.concatenate((X_train, X_train_id.toarray()), axis=1)
X_test = np.concatenate((X_test, X_test_id.toarray()), axis=1)

    
regr = Ridge(alpha=1.0)
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)
    

In [15]:
submission = pd.DataFrame(test[["test_id"]])
submission['price'] = np.expm1(y_pred)
submission.to_csv("submission.csv", index=False)